In [13]:
#------------------- Change working directory to project root -------------------#
from pathlib import Path
import os

cur = Path().resolve()
while not (cur / "src").is_dir():
    if cur == cur.parent: raise RuntimeError("No 'src' dir")
    cur = cur.parent

os.chdir(cur)
print(f"[INFO] Changed working directory to project root: {cur}")

[INFO] Changed working directory to project root: C:\Users\stefa\OneDrive\Documentos\LoRaPriv


In [14]:
# -------------------------------------- External Libraries --------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import importlib.util
# ----------------------------------------- Local Imports ----------------------------------------
from src.core                  import LoRaPhyParams, LoRaFrameParams
from src.codec                 import LoRaCodec
from src.mod                   import LoRaModulator, plot_frame
from src.demod                 import LoRaDemodulator, plot_demodulation

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
# ── Testing functions ----------------------------------------------------
def test_config(sf, bw, spc):

    
    payload = list(range(1 << sf))  # All possible symbols from 0 to 2^sf - 1

    phy = LoRaPhyParams(spreading_factor=sf, bandwidth=bw, samples_per_chip=spc)

    frame = LoRaFrameParams(preamble_symbol_count=8, explicit_header=True, sync_word=0x00)


    mod = LoRaModulator(phy, frame, backend="numpy")
    signal = mod.modulate(payload, include_frame=False)

    has_cupy = importlib.util.find_spec("cupy") is not None
    backend = "cupy" if has_cupy else "numpy"

    demod = LoRaDemodulator(phy, backend=backend, fold_mode="CPA")
    received = demod.demodulate(signal)

    correct = sum(1 for a, b in zip(payload, received) if a == b)
    likelihood = correct / len(payload) * 100

    demod.backend.clear_memory()  # Clear GPU memory if used
    
    return payload, received.tolist(), likelihood


def run_tests(configs, group_by="sf", ncols=3):
    """
    Runs demodulation tests across a range of LoRa configurations,
    grouping and plotting results per group_by field.
    
    :param configs: Range of LoRa configurations to test, each defined as a tuple (sf, bw, spc).
    :type configs: List of (sf, bw, spc) tuples
    
    :group_by: Field to group results by, one of 'sf', 'bw', 'spc'.
    :type group_by: str, default 'sf'

    :ncols: Number of subplot columns per group.
    :type ncols: int, default 3

    """
    groups = defaultdict(list)
    for sf, bw, spc in configs:
        key = {"sf": sf, "bw": bw, "spc": spc}.get(group_by, "default")
        groups[key].append((sf, bw, spc))

    for key, cfgs in groups.items():
        n = len(cfgs)
        nrows = int(np.ceil(n / ncols))
        fig, axs = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
        axs = np.atleast_1d(axs).flatten()

        for ax, (sf, bw, spc) in zip(axs, cfgs):
            payload, received, accuracy = test_config(sf, bw, spc)
            payload = np.array(payload)
            received = np.array(received)

            # Ensure equal length
            min_len = min(len(payload), len(received))
            payload = payload[:min_len]
            received = received[:min_len]

            # Compare and plot
            correct = (payload == received)
            colors = np.where(correct, "green", "red")
            ax.scatter(payload, received, c=colors, s=15)
            
            ax.set_title(f"sf={sf}, spc={spc}, bw={bw/1e3:.0f}kHz\nSymbol Match Rate= {accuracy:.1f}%", fontsize=10)
            ax.set_xlabel("Transmitted Symbol")
            ax.set_ylabel("Demodulated Symbol")
            ax.grid(True, linestyle='--', alpha=0.5)
            ax.set_xticks([])

        for j in range(n, len(axs)):
            axs[j].axis("off")

        fig.suptitle(f"Grouped by {group_by}: {key}", fontsize=14, fontweight="bold")
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()




# Evaluación cualitativa de constelaciones de símbolos en LoRa

El siguiente código realiza una prueba cualitativa sobre el proceso completo de modulación y demodulación en distintas configuraciones de LoRa (combinaciones de factor de dispersión, ancho de banda y sobremuestreo). El objetivo es verificar que **toda la constelación de símbolos pueda ser correctamente transmitida y recuperada** sin errores, bajo condiciones ideales de canal.

Para cada configuración, se transmiten todos los símbolos posibles y se comparan con los demodulados. El resultado es un gráfico que muestra los símbolos enviados frente a los recibidos, permitiendo validar visualmente la corrección del sistema de extremo a extremo.


In [ ]:
configs = [
    (7, 125e3, 1), (7, 250e3, 2), 
    (8, 125e3, 1), (8, 250e3, 2),
    (9, 125e3, 1), (9, 250e3, 2), 
    (10, 125e3, 1), (10, 250e3, 2),
    (11, 125e3, 1), (11, 250e3, 2),
    (12, 125e3, 1), (12, 250e3, 2),
]

try:
    run_tests(configs, group_by="sf", ncols=3)
except MemoryError:
    print("OutOfMemoryError: The test may require more memory than available.")

NameError: name 'run_tests' is not defined